## Kiwi 토크나이저를 이용한 BM25Retriever

In [2]:
# 필요한 라이브러리 설치
# !pip install -qU kiwipiepy konlpy langchain-teddynote


[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: pip install --upgrade pip


깔끔하게 출력결과를 확인하기 위한 함수를 정의합니다.

In [12]:
def pretty_print(docs):
    for i, doc in enumerate(docs):
        if "score" in doc.metadata:
            print(f"[{i+1}] {doc.page_content} ({doc.metadata['score']:.4f})")
        else:
            print(f"[{i+1}] {doc.page_content}")

In [13]:
# 비교를 위한 BM25Retriever
from langchain_community.retrievers import BM25Retriever

# 커스텀 구현한 한국어 형태소 분석기(Kiwi)를 사용한 BM25Retriever
from langchain_teddynote.retrievers import KiwiBM25Retriever

sample_texts = [
    "금융보험은 장기적인 자산 관리와 위험 대비를 목적으로 고안된 금융 상품입니다.",
    "금융저축산물보험은 장기적인 저축 목적과 더불어, 축산물 제공 기능을 갖추고 있는 특별 금융 상품입니다.",
    "금융보씨 험한말 좀 하지마시고, 저축이나 좀 하시던가요. 뭐가 그리 급하신지 모르겠네요.",
    "금융단폭격보험은 저축은 커녕 위험 대비에 초점을 맞춘 상품입니다. 높은 위험을 감수하고자 하는 고객에게 적합합니다.",
]

In [14]:
# 리트리버를 생성합니다.
kiwi = KiwiBM25Retriever.from_texts(sample_texts)

In [15]:
# 유사도 검색을 수행합니다.
pretty_print(kiwi.invoke("금융보험"))

[1] 금융보험은 장기적인 자산 관리와 위험 대비를 목적으로 고안된 금융 상품입니다.
[2] 금융저축산물보험은 장기적인 저축 목적과 더불어, 축산물 제공 기능을 갖추고 있는 특별 금융 상품입니다.
[3] 금융단폭격보험은 저축은 커녕 위험 대비에 초점을 맞춘 상품입니다. 높은 위험을 감수하고자 하는 고객에게 적합합니다.
[4] 금융보씨 험한말 좀 하지마시고, 저축이나 좀 하시던가요. 뭐가 그리 급하신지 모르겠네요.


개인적인 필요에 의하여 유사도 점수를 계산하고 metadata 에 `score` 를 추가하는 기능을 추가하였습니다.

In [16]:
# 유사도 검색을 수행합니다.
pretty_print(kiwi.search_with_score("금융보험"))

[1] 금융보험은 장기적인 자산 관리와 위험 대비를 목적으로 고안된 금융 상품입니다. (0.2673)
[2] 금융저축산물보험은 장기적인 저축 목적과 더불어, 축산물 제공 기능을 갖추고 있는 특별 금융 상품입니다. (0.2616)
[3] 금융단폭격보험은 저축은 커녕 위험 대비에 초점을 맞춘 상품입니다. 높은 위험을 감수하고자 하는 고객에게 적합합니다. (0.2463)
[4] 금융보씨 험한말 좀 하지마시고, 저축이나 좀 하시던가요. 뭐가 그리 급하신지 모르겠네요. (0.2248)


In [18]:
bm25 = BM25Retriever.from_texts(sample_texts)

# BM25 KiwiBM25 비교
print(f'Kiwi:  \t {kiwi.invoke("금융보험")[0].page_content}')
print(f'BM25:  \t {bm25.invoke("금융보험")[0].page_content}')

Kiwi:  	 금융보험은 장기적인 자산 관리와 위험 대비를 목적으로 고안된 금융 상품입니다.
BM25:  	 금융단폭격보험은 저축은 커녕 위험 대비에 초점을 맞춘 상품입니다. 높은 위험을 감수하고자 하는 고객에게 적합합니다.


k 값 설정

In [19]:
# 리트리버를 생성합니다.
kiwi = KiwiBM25Retriever.from_texts(sample_texts)
kiwi.k = 2
# 유사도 검색을 수행합니다.
pretty_print(kiwi.search_with_score("금융보험"))

[1] 금융보험은 장기적인 자산 관리와 위험 대비를 목적으로 고안된 금융 상품입니다. (0.2673)
[2] 금융저축산물보험은 장기적인 저축 목적과 더불어, 축산물 제공 기능을 갖추고 있는 특별 금융 상품입니다. (0.2616)


## KonlPy(Kkma, Okt) 사용한 BM25Retriever

In [21]:
# 커스텀 구현한 한국어 형태소 분석기를 사용한 BM25Retriever
from langchain_teddynote.retrievers import (
    KkmaBM25Retriever,
    OktBM25Retriever,
)

In [22]:
# 리트리버를 생성합니다.
kkma = KkmaBM25Retriever.from_texts(sample_texts)
okt = OktBM25Retriever.from_texts(sample_texts)

In [23]:
# kkma + BM25
pretty_print(kkma.invoke("금융보험"))

[1] 금융보험은 장기적인 자산 관리와 위험 대비를 목적으로 고안된 금융 상품입니다.
[2] 금융저축산물보험은 장기적인 저축 목적과 더불어, 축산물 제공 기능을 갖추고 있는 특별 금융 상품입니다.
[3] 금융보씨 험한말 좀 하지마시고, 저축이나 좀 하시던가요. 뭐가 그리 급하신지 모르겠네요.
[4] 금융단폭격보험은 저축은 커녕 위험 대비에 초점을 맞춘 상품입니다. 높은 위험을 감수하고자 하는 고객에게 적합합니다.


In [24]:
# 유사도 검색을 수행합니다.
pretty_print(kkma.search_with_score("금융보험"))

[1] 금융보험은 장기적인 자산 관리와 위험 대비를 목적으로 고안된 금융 상품입니다. (0.2770)
[2] 금융저축산물보험은 장기적인 저축 목적과 더불어, 축산물 제공 기능을 갖추고 있는 특별 금융 상품입니다. (0.2701)
[3] 금융보씨 험한말 좀 하지마시고, 저축이나 좀 하시던가요. 뭐가 그리 급하신지 모르겠네요. (0.2268)
[4] 금융단폭격보험은 저축은 커녕 위험 대비에 초점을 맞춘 상품입니다. 높은 위험을 감수하고자 하는 고객에게 적합합니다. (0.2260)


In [25]:
# okt + BM25
pretty_print(okt.invoke("금융보험"))

[1] 금융보험은 장기적인 자산 관리와 위험 대비를 목적으로 고안된 금융 상품입니다.
[2] 금융저축산물보험은 장기적인 저축 목적과 더불어, 축산물 제공 기능을 갖추고 있는 특별 금융 상품입니다.
[3] 금융단폭격보험은 저축은 커녕 위험 대비에 초점을 맞춘 상품입니다. 높은 위험을 감수하고자 하는 고객에게 적합합니다.
[4] 금융보씨 험한말 좀 하지마시고, 저축이나 좀 하시던가요. 뭐가 그리 급하신지 모르겠네요.


In [26]:
# 유사도 검색을 수행합니다.
pretty_print(okt.search_with_score("금융보험"))

[1] 금융보험은 장기적인 자산 관리와 위험 대비를 목적으로 고안된 금융 상품입니다. (0.2672)
[2] 금융저축산물보험은 장기적인 저축 목적과 더불어, 축산물 제공 기능을 갖추고 있는 특별 금융 상품입니다. (0.2620)
[3] 금융단폭격보험은 저축은 커녕 위험 대비에 초점을 맞춘 상품입니다. 높은 위험을 감수하고자 하는 고객에게 적합합니다. (0.2460)
[4] 금융보씨 험한말 좀 하지마시고, 저축이나 좀 하시던가요. 뭐가 그리 급하신지 모르겠네요. (0.2248)
